# Пример 01: Быстрый чат через OpenAI SDK

Этот ноутбук демонстрирует, как использовать OpenAI SDK с Microsoft Foundry Local для быстрых взаимодействий в чате.

## Обзор

Этот пример показывает:
- Использование OpenAI Python SDK с Foundry Local
- Работа с конфигурациями Azure OpenAI и локального Foundry
- Реализация корректной обработки ошибок и стратегий резервирования
- Использование FoundryLocalManager для управления сервисами


## Предварительные условия

Убедитесь, что у вас:
1. Установлен и запущен Foundry Local
2. Загружена модель (например, `foundry model run phi-4-mini`)
3. Установлены необходимые пакеты Python

### Установка зависимостей


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## Импорт необходимых библиотек


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## Конфигурация

Настройте вашу конфигурацию. Вы можете использовать один из следующих вариантов:
1. **Azure OpenAI** (облачный сервис)
2. **Foundry Local** (локальное выполнение)

### Вариант 1: Конфигурация Azure OpenAI


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### Вариант 2: Локальная конфигурация Foundry


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## Инициализация клиента

Создайте клиент OpenAI на основе вашей конфигурации:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## Пример простого чата

Давайте протестируем простое взаимодействие в чате:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## Интерактивная сессия чата

Попробуйте различные типы запросов, чтобы увидеть, как модель реагирует:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## Расширенное использование: потоковые ответы

Для более длинных ответов потоковая передача может обеспечить лучший пользовательский опыт:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## Обработка ошибок и диагностика

Давайте проверим статус сервиса и доступные модели:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## Тестирование пользовательских запросов

Используйте ячейку ниже, чтобы протестировать свои собственные запросы:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## Резюме

В этом ноутбуке было показано:

1. **✅ Настройка клиента**: Как настроить OpenAI SDK с Foundry Local
2. **✅ Базовый чат**: Простые взаимодействия запрос-ответ
3. **✅ Потоковая передача**: Генерация ответов в реальном времени
4. **✅ Обработка ошибок**: Надежная обработка ошибок и диагностика
5. **✅ Состояние сервиса**: Проверка доступности модели и статуса сервиса

### Следующие шаги

- **Пример 02**: Расширенная интеграция SDK с поддержкой Azure OpenAI
- **Пример 04**: Создание чат-приложения с Chainlit
- **Пример 05**: Системы оркестрации с несколькими агентами
- **Пример 06**: Интеллектуальная маршрутизация моделей

### Основные преимущества Foundry Local

- 🔒 **Конфиденциальность**: Данные никогда не покидают ваше устройство
- ⚡ **Скорость**: Низкая задержка при локальном выполнении
- 💰 **Стоимость**: Отсутствие затрат на использование API
- 🔌 **Оффлайн**: Работает без подключения к интернету
- 🛠️ **Совместимость**: API, совместимый с OpenAI
